In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import pickle

transactions = pd.read_csv('../transactions.csv', index_col='transactionId')

In [ ]:
def m_accuracy(model, X, y):
    '''
    returns: the accuracy of the given model, on the digits dataset
    '''
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
    clf = model
    clf.fit(X_train, y_train)
    return clf.score(X_test,y_test)

def m_accuracy_mv(model, X, y, N:int):
    '''
    returns: the tupple (mean, std_var)
    '''

    accuracies = [ m_accuracy(model, X, y) for i in range(N) ]
    return (np.mean(accuracies), np.std(accuracies))

def norm2float(df:pd.DataFrame, cols:list):
    for col in cols:
        df[col] = df[col].str.replace(',', '.').astype(float)
        
    return df

In [6]:
transactions.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
transactionId,,,,,,,,,,
0,1,3,9839.64,1.123101e+10,170136.00,160296.36,2.197979e+10,0.0,0.00,0
1,1,3,1864.28,1.166654e+10,21249.00,19384.72,2.204428e+10,0.0,0.00,0
2,1,4,181.00,1.130549e+10,181.00,0.00,1.553264e+09,0.0,0.00,1
3,1,1,181.00,1.840084e+09,181.00,0.00,1.389970e+08,21182.0,0.00,1
4,1,3,11668.14,1.204854e+10,41554.00,29885.86,2.123070e+10,0.0,0.00,0
5,1,3,7817.71,1.900456e+08,53860.00,46042.29,2.573487e+09,0.0,0.00,0
6,1,3,7107.77,1.154989e+09,183195.00,176087.23,2.408069e+09,0.0,0.00,0
7,1,3,7861.64,1.191285e+10,176087.23,168225.59,2.633326e+09,0.0,0.00,0
8,1,3,4024.36,1.126501e+10,2671.00,0.00,2.117693e+10,0.0,0.00,0


In [7]:
X = transactions.drop(columns=['isFraud'])
y = transactions['isFraud']
smote = SMOTE(random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [8]:
# Model training and testing
N = 10
df = pd.DataFrame(data=[], columns=['n_neighbors', 'accuracy', 'smote_accuracy'])

for i in range(2, N):
    clf = KNeighborsClassifier(n_neighbors=i)
    smote_clf = KNeighborsClassifier(n_neighbors=i)
    
    clf.fit(X_train, y_train)
    smote_clf.fit(X_train_resampled, y_train_resampled)

    df.loc[len(df)] = [int(i), clf.score(X_test, y_test), smote_clf.score(X_test, y_test)]

df.head(15)

,n_neighbors,accuracy,smote_accuracy
0,2.0,0.992085,0.922809
1,3.0,0.991921,0.872914
2,4.0,0.992145,0.885149
3,5.0,0.992145,0.847721
4,6.0,0.992154,0.860263
5,7.0,0.992154,0.830443
6,8.0,0.992154,0.843243
7,9.0,0.992154,0.817608


In [ ]:
plt.plot(df['n_neighbors'], df['accuracy'], color='blue')
plt.plot(df['n_neighbors'], df['smote_accuracy'], color='red')
plt.show()

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
knn_clf.fit(X_train, y_train)

pickle.dump(knn_clf, open('./KNN.pkl', 'wb'))